In [2]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir
from os.path import join
from sklearn import metrics
import os

In [3]:
local_dir = os.path.join("/DATA",os.environ.get("USER"),"MNIST_data")
os.makedirs(local_dir,mode=0o755, exist_ok=True)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(local_dir, one_hot=True)

Extracting /DATA/solomiia.kurchaba/MNIST_data/train-images-idx3-ubyte.gz
Extracting /DATA/solomiia.kurchaba/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /DATA/solomiia.kurchaba/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /DATA/solomiia.kurchaba/MNIST_data/t10k-labels-idx1-ubyte.gz


Przygotowujemy dane i sieć zgodnie z opisem w pliku MNIST RNN

(jedyna różnica, tym razem nie potrzebujemy zbioru testowego)

In [5]:
num_input = 28 
time_steps = 28 
hidden = 128
output_classes = 10
lr=0.0001
batch_size=128
x_tr, y_batches = mnist.train.next_batch(batch_size)
x_batches = x_tr.reshape(-1, time_steps, num_input)


In [35]:
from tensorflow.contrib import rnn
tf.reset_default_graph()
tf.set_random_seed=10
X=tf.placeholder(tf.float32,[None,time_steps,num_input])
Y=tf.placeholder(tf.float32,[None,output_classes])
x=tf.unstack(X,time_steps,1)
basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden) # here the first set of weights and biases is defined
rnn_output, states=rnn.static_rnn(basic_cell,x,dtype=tf.float32)
stacked_rnn_output=tf.reshape(rnn_output[-1],[-1,hidden])
stacked_outputs=tf.layers.dense(stacked_rnn_output,output_classes)# the output layer. (+additional set of weights and biases)
predict=tf.nn.softmax(stacked_outputs)
y_pred=tf.argmax(predict,1)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=Y))

optimizer=tf.train.AdamOptimizer(learning_rate=lr)
training_op=optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(predict, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init=tf.global_variables_initializer()


Ładujemy obrazek, wcześniej narysowany w Paint, w formacie np.ndarray, oraz ręcznie przygotowujemy label do obrazka, tak podanie poprawnych danych wyjściowych jest obowiązkowe. Zmieniamy wymiar obrazku oraz lablu na zgodny z wymaganiami sieci.

In [7]:
from scipy import misc
im=misc.imread("two.png",mode='L')
l=[0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0]
lbl=np.array(l)
lbl=lbl.reshape(-1,10)
x_n=im.reshape(-1,time_steps,num_input)

In [24]:
from IPython.display import Image
im=Image(filename='two.png')
im

In [16]:
np.shape(lbl)

(1, 10)

Aktywacja cieci. Zamiast danych testowych ładujemy nasz obrazek. Na końcu wyświetlamy procent poprawnych odpowiedzi uzyskanych w ostatnim kroku iteracji treningowej, lable testowego obrazku, przewidziany przez sieć, oraz wynik na podstawie którego sieć robi wniosek że to trójka (wyjście z sieci po działaniu softmaxa)

In [36]:
epochs=2500
with  tf.Session() as sess:
    init.run()
    for ep in range(epochs):
        sess.run(training_op,feed_dict={X: x_batches, Y: y_batches})
        loss=loss_op.eval(feed_dict={X: x_batches, Y: y_batches})
        if ep % 100 == 0:
            loss=loss_op.eval(feed_dict={X: x_batches, Y: y_batches})
            print(ep,"\tloss:",loss)
    train_accuracy=accuracy.eval(feed_dict={X: x_batches, Y: y_batches})  
    print(train_accuracy)
    y_pred=y_pred.eval(feed_dict={X: x_n, Y: lbl})
    print(y_pred)
    predict=predict.eval(feed_dict={X: x_n, Y: lbl})
   
    print(predict)
    
        
        
sess.close()   

0 	loss: 2.29418
100 	loss: 1.99204
200 	loss: 1.80027
300 	loss: 1.63319
400 	loss: 1.56301
500 	loss: 1.53374
600 	loss: 1.5202
700 	loss: 1.5151
800 	loss: 1.51263
900 	loss: 1.5111
1000 	loss: 1.51005
1100 	loss: 1.5093
1200 	loss: 1.50874
1300 	loss: 1.5083
1400 	loss: 1.50795
1500 	loss: 1.5076
1600 	loss: 1.50034
1700 	loss: 1.49999
1800 	loss: 1.49976
1900 	loss: 1.49958
2000 	loss: 1.49942
2100 	loss: 1.49926
2200 	loss: 1.4919
2300 	loss: 1.49162
2400 	loss: 1.49115
0.984375
[3]
[[  4.80813952e-03   3.73418964e-02   6.12823144e-02   8.23082745e-01
    9.70509354e-05   4.67024855e-02   7.42168492e-03   2.41419976e-03
    1.00439526e-02   6.80556148e-03]]
